In [2]:
%load_ext autoreload
%autoreload 2

In [20]:
import matplotlib.pyplot as plt
import matplotlib as mpl
from cycler import cycler
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
import numpy as np
import scipy.stats as sps
import json, pickle, os, corner, glob, sys
import enterprise.signals.utils as utils
from enterprise.pulsar import Pulsar
import arviz as az
from collections import defaultdict
import pandas as pd

In [4]:
import la_forge.diagnostics as dg
import la_forge.core as co
from la_forge.rednoise import plot_rednoise_spectrum, plot_free_spec
from la_forge.utils import epoch_ave_resid

In [5]:
color_cycle_wong = ['#000000','#E69F00','#009E73','#56B4E9','#0072B2','#F0E442','#D55E00','#CC79A7']
mpl.rcParams['axes.prop_cycle'] = cycler(color=color_cycle_wong)

In [6]:
current_path = os.getcwd()
splt_path = current_path.split("/")
top_path_idx = splt_path.index("akaiser")
#top_path_idx = splt_path.index("nanograv")
top_dir = "/".join(splt_path[0 : top_path_idx + 1])

In [7]:
e_e_path = top_dir + "/enterprise_extensions/"
sys.path.insert(0, e_e_path)
from enterprise_extensions.timing_2 import get_astrometric_priors
import nltm_plot_utils_v2 as nltm

In [29]:
psr_name = 'J0740+6620'#'J1640+2224'#'J2317+1439' 'J1909-3744'
psr_dir = current_path + '/{}/chains/'.format(psr_name)
psrlist = [psr_name]
datareleases = ['11yr','12p5yr']

with open(psr_dir+'12p5yr/orig_timing_pars.pkl', 'rb') as fin:
    tm_params_orig = pickle.load(fin)
#psrs = nltm.get_psrs(psrlist,datareleases)
#pardict = nltm.get_pardict(psrs,datareleases)

In [30]:
for ds in tm_params_orig.keys():
    print(ds,":  ",tm_params_orig[ds])

ELONG :   (1.8109385467972050066, 2.1103251470945240098e-10)
ELAT :   (0.76973350844883281496, 2.2311349392271817599e-10)
F0 :   (346.53199647626473734, 3.5380642677468050184e-13)
F1 :   (-1.4638934308892426497e-15, 9.2435572524311821995e-21)
PMELONG :   (-2.7356846634113307637, 0.012142356227228930099)
PMELAT :   (-32.48343129713580879, 0.018648345011703794383)
PX :   (0.81384822250111943376, 0.14889809453013924667)
SINI :   (0.9991467049877268414, 9.439620679985960788e-05)
PB :   (4.7669446192854074695, 8.663503206659659445e-11)
A1 :   (3.9775561438826598781, 8.404182888684216287e-08)
PBDOT :   (9.694336924172837473e-13, 1.8324710937317905573e-13)
TASC :   (57685.55769342061103, 1.4569614009629760555e-08)
EPS1 :   (-5.6836442142772590543e-06, 2.2150268317828595291e-08)
EPS2 :   (-1.844356373920329148e-06, 1.5256729094220466134e-08)
M2 :   (0.24848467236967541893, 0.004228455950746962816)
FD1 :   (-1.4449508042858721631e-05, 1.9009233344219203741e-07)
DMX_0001 :   (-0.0009941889965953

In [31]:
chaindir_list = []
#indices = np.arange(1,6,1)
indices = [1,2]
ephems = ['DE436']
#big_chaindir = os.getcwd()+'/chains/{}_all_var'.format(psr)
#chaindir_list.append(os.getcwd()+'/{}/chains/{}/{}_uniform_DE436_tm_{}'.format(psr_name,'11yr',psr_name,1))

for i in indices:
    for datarelease in datareleases:
        if i == 1:
            chaindir_list.append(psr_dir+'{}/{}_uniform_DE436_tm_{}'.format(datarelease,psr_name,i))
        elif i == 2 and datarelease == '12p5yr':
            chaindir_list.append(psr_dir+'{}/{}_uniform_DE436_nltm_ltm_1_core'.format(datarelease,psr_name))
            chaindir_list.append(psr_dir+'{}/{}_uniform_DE436_nltm_ltm_2_core'.format(datarelease,psr_name))
            chaindir_list.append(psr_dir+'{}/{}_uniform_DE436_nltm_ltm_4_core'.format(datarelease,psr_name))

[print(chaindir) for chaindir in chaindir_list];

/hyrule/data/users/akaiser/enterprise_timing/J0740+6620/chains/11yr/J0740+6620_uniform_DE436_tm_1
/hyrule/data/users/akaiser/enterprise_timing/J0740+6620/chains/12p5yr/J0740+6620_uniform_DE436_tm_1
/hyrule/data/users/akaiser/enterprise_timing/J0740+6620/chains/12p5yr/J0740+6620_uniform_DE436_nltm_ltm_1_core
/hyrule/data/users/akaiser/enterprise_timing/J0740+6620/chains/12p5yr/J0740+6620_uniform_DE436_nltm_ltm_2_core
/hyrule/data/users/akaiser/enterprise_timing/J0740+6620/chains/12p5yr/J0740+6620_uniform_DE436_nltm_ltm_4_core


In [32]:
chaindir_core_1 = psr_dir+'12p5yr/{}_uniform_DE436_nltm_ltm_1_core'.format(psr_name)
chaindir_core_2 = psr_dir+'12p5yr/{}_uniform_DE436_nltm_ltm_2_core'.format(psr_name)
chaindir_core_4 = psr_dir+'12p5yr/{}_uniform_DE436_nltm_ltm_4_core'.format(psr_name)

In [33]:
chain_dict = nltm.get_chain_tmparam_dict(chaindir_list)

11yr J0740+6620_uniform_DE436_tm_1 Loaded.      
12p5yr J0740+6620_uniform_DE436_tm_1 Loaded.      
12p5yr J0740+6620_uniform_DE436_nltm_ltm_1_core Loaded.      
12p5yr J0740+6620_uniform_DE436_nltm_ltm_2_core Loaded.      
12p5yr J0740+6620_uniform_DE436_nltm_ltm_4_core Loaded.      


In [34]:
for p in chain_dict.keys():
    print(p)
    for px_p in chain_dict[p].keys():
        print('  ',px_p)
        for dset in chain_dict[p][px_p].keys():
            print('    ',dset)
            for subset in chain_dict[p][px_p][dset]['chains']:
                print('      ',np.shape(subset))

uniform
   parfile_priors_DE436
     11yr
       (71175, 53)
     12p5yr
       (38850, 72)
       (8100, 258)
       (7875, 258)
       (7500, 258)


In [35]:
trimmed_chain_dict = nltm.get_trimmed_chain_tmparam_dict(chain_dict)

In [36]:
for p in trimmed_chain_dict.keys():
    print(p)
    for px_p in trimmed_chain_dict[p].keys():
        print('  ',px_p)
        for dset in trimmed_chain_dict[p][px_p].keys():
            print('    ',dset)
            for subset in trimmed_chain_dict[p][px_p][dset]['chains']:
                print('      ',np.shape(subset))

uniform
   parfile_priors_DE436
     11yr
       (7500, 53)
     12p5yr
       (7500, 72)
       (7500, 258)
       (7500, 258)
       (7500, 258)


# Make a List of `la_forge` Cores

In [37]:
core_list = []
for chaindir in chaindir_list:
    core_list.append(co.Core(label='_'.join(chaindir.split('/')[-2:]), chaindir=chaindir))

Appending PTMCMCSampler sampling parameters to end of parameter list.
If unwanted please provide a parameter list.
No burn specified. Burn set to 25% of chain length, 23725
You may change the burn length with core.set_burn()
Red noise frequencies must be set before plotting most red noise figures.
Please use core.set_rn_freqs() to set, if needed.
Appending PTMCMCSampler sampling parameters to end of parameter list.
If unwanted please provide a parameter list.
No burn specified. Burn set to 25% of chain length, 12950
You may change the burn length with core.set_burn()
Red noise frequencies must be set before plotting most red noise figures.
Please use core.set_rn_freqs() to set, if needed.
Appending PTMCMCSampler sampling parameters to end of parameter list.
If unwanted please provide a parameter list.
No burn specified. Burn set to 25% of chain length, 2700
You may change the burn length with core.set_burn()
Red noise frequencies must be set before plotting most red noise figures.
Plea

```
standard_chain = pd.read_csv(os.getcwd()+'/standard_runs/chain_1.txt', sep='\t', dtype=float, header=None).values
par_names = ['J1744-1134_Rcvr1_2_GASP_efac','J1744-1134_Rcvr1_2_GASP_log10_ecorr',
             'J1744-1134_Rcvr1_2_GASP_log10_equad','J1744-1134_Rcvr1_2_GUPPI_efac',
             'J1744-1134_Rcvr1_2_GUPPI_log10_ecorr', 'J1744-1134_Rcvr1_2_GUPPI_log10_equad',
             'J1744-1134_Rcvr_800_GASP_efac', 'J1744-1134_Rcvr_800_GASP_log10_ecorr',
             'J1744-1134_Rcvr_800_GASP_log10_equad', 'J1744-1134_Rcvr_800_GUPPI_efac',
             'J1744-1134_Rcvr_800_GUPPI_log10_ecorr', 'J1744-1134_Rcvr_800_GUPPI_log10_equad',
             'J1744-1134_red_noise_gamma', 'J1744-1134_red_noise_log10_A',
             'lnlike', 'lnprior', 'chain_accept', 'pt_chain_accept']
standard_cores = co.Core(label='standard runs',chain = standard_chain,params=par_names)
```

In [ ]:
dset = '11yr'
por = 'uniform'
px_por = 'parfile_priors_DE436'
tmp_chaindir_indices = nltm.get_chaindir_indices(chaindir_list)
print(tmp_chaindir_indices.keys())
print(tmp_chaindir_indices[por].keys())
print(tmp_chaindir_indices[por][px_por].keys())

In [ ]:
#tmp_core_list.append(standard_cores)
#tmp_core_labels.append('standard runs')
tmp_core_common_pars = []
tmp_core_noncommon_pars = []
tmp_core_tm_pars = []

for core in core_list:
    if len(tmp_core_common_pars)==0:
        for core_param in core.params:
            if core_param not in ['lnlike','lnprior', 'chain_accept', 'pt_chain_accept', 'chain accept', 'pt chain accept']:
                tmp_core_common_pars.append(core_param)
            if 'timing' in core_param.split('_'):
                tmp_core_tm_pars.append(core_param)
    else:
        noncommon_pars = []
        for com_param in tmp_core_common_pars:
            if com_param not in core.params:
                noncommon_pars.append(com_param)
        for ncom_param in noncommon_pars:
            del tmp_core_common_pars[tmp_core_common_pars.index(ncom_param)]
        for param in core.params:
            if param not in tmp_core_common_pars:
                tmp_core_noncommon_pars.append(param)

In [ ]:
print(tmp_core_common_pars)
print(tmp_core_noncommon_pars)

In [ ]:
tmp_core_kwargs = {'hist':True, 'ncols':3, 'title_y':1.01,
                   'hist_kwargs':dict(fill=False),'linewidth':3.}
dg.plot_chains(core_list,
               suptitle=psr_name,
               pars = tmp_core_common_pars,
               **tmp_core_kwargs)
#,save='Figures/J1744_Noise_comp.png')

```
dataset='11yr'
priors = ['uniform']
px_priors = ['parfile_priors_DE436']
dg.plot_chains(tmp_core_list[:-1],
               suptitle='{} {} {} Runs Timing Parameters'.format(psrlist[0],dataset,px_priors[0]),
               legend_labels=tmp_core_labels[:-1],
               pars = tmp_core_tm_pars,linewidth=3.,
               title_y=1.03,legend_loc=(.4,.8),ncols=2)
#,save='Figures/J1744_tm_comp.png')
```

In [ ]:
for tmp_core,tmp_chaindir in zip(core_list,chaindir_list):
    dg.plot_chains(tmp_core,pars=['lnlike'],hist=False,
                   suptitle=tmp_chaindir.split('/')[-1].split('_')[0]+' '+tmp_chaindir.split('/')[-2],
                   ncols=1,title_y=1.05)

In [ ]:
titles = []
for param in tmp_core.params:
    if 'timing' in param.split('_'):
        if 'DMX' in param.split('_'):
            titles.append(('_').join(param.split('_')[-2:]))
        else:
            titles.append(param.split('_')[-1])
    else:
        if psr_name in param.split('_'):
            titles.append((' ').join(param.split('_')[1:]))
        else:
            titles.append(param)

In [ ]:
print(titles)

In [ ]:
for tmp_core,tmp_chaindir in zip(core_list,chaindir_list):
    titles = []
    for param in tmp_core.params:
        if 'timing' in param.split('_'):
            if 'DMX' in param.split('_'):
                titles.append(('_').join(param.split('_')[-2:]))
            else:
                titles.append(param.split('_')[-1])
        else:
            if psr_name in param.split('_'):
                titles.append((' ').join(param.split('_')[1:]))
            else:
                titles.append(param)
    print('/hyrule/data/users/akaiser/enterprise_timing/Figures/{}_{}_runs.png'.format(psr_name,tmp_chaindir.split('/')[-2]))
    dg.plot_chains(tmp_core,pars=tmp_core.params,
                   suptitle=tmp_chaindir.split('/')[-1].split('_')[0]+' '+tmp_chaindir.split('/')[-2],
                   ncols=3,title_y=1.01,titles=titles,hist=True,
                   save='/hyrule/data/users/akaiser/enterprise_timing/Figures/{}_{}_runs.png'.format(psr_name,tmp_chaindir.split('/')[-2]))

# Plot Cores with `la_forge`

In [ ]:
def plt_selection_of_dataset(dataset,px_prior='all'):
    if px_prior == 'all':
        selection = ['vlbi','parfile']
    else:
        selection = [px_prior]

    core_kwargs = {'hist':True, 'ncols':3, 'title_y':1.01,
                   'suptitle':'{} {} {} Runs'.format(psrlist[0],dataset,px_prior),
                   'bins':10,'hist_kwargs':dict(fill=False),'linewidth':2.}
                   #'save':'Figures/{}_{}_runs.png'.format(psrlist[0],dataset)}
    misc_kwargs = {'legend_on':True,'legend_loc':(.7,0.05)}
    priors = []
    px_priors = []
    for p in chain_dict.keys():
        for px_p in chain_dict[p].keys():
            for select in selection:
                if select in px_p.split('_'):
                    if dataset in chain_dict[p][px_p].keys():
                        if p not in priors:
                            priors.append(p)
                        if px_p not in px_priors:
                            px_priors.append(px_p)
    if priors and px_priors:
        nltm.plot_common_chains(core_list,chaindir_list,priors,px_priors,dataset,
                                plot_kwargs=core_kwargs,misc_kwargs=misc_kwargs)
    else:
        print(selection,'not in',dataset)

## 11yr Cores

In [ ]:
#All Runs
plt_selection_of_dataset('11yr')

## 12.5yr Cores

In [ ]:
#All Runs
plt_selection_of_dataset('12p5yr')

# Do Plotting with `arviz`
Making `Arviz` dictionaries to plot all chains

In [ ]:
az_comb,az_comb_chain_dict = nltm.get_combined_arviz_obj_from_dict(trimmed_chain_dict,psrs,return_dict=True)

In [ ]:
#print(az_comb.posterior)

In [ ]:
#az.summary(az_comb)

In [ ]:
az.plot_trace(az_comb,plot_kwargs={'linewidth':3});

In [ ]:
white_pars_all = []
dmx_pars_all = []
parfile_priors_pars_all = []
for param in az_comb_chain_dict.keys():
    if param not in ['lnlike','lnprior', 'chain accept', 'pt chain accept']:
        if param.split('_')[-1] in ['efac','equad','ecorr']:
            white_pars_all.append(param)
        elif 'DMX' in param.split('_'):
            dmx_pars_all.append(param)
        else:
            parfile_priors_pars_all.append(param)

White noise parameters

In [ ]:
az.plot_trace(az_comb,var_names=white_pars_all,plot_kwargs={'linewidth':3});

DMX Parameters

In [ ]:
az.plot_trace(az_comb,var_names=dmx_pars_all,plot_kwargs={'linewidth':3});

Red + Timing Parameters

In [ ]:
az.plot_trace(az_comb,var_names=parfile_priors_pars_all,plot_kwargs={'linewidth':3});

# Rescaling the timing parameters to their physical values to properly compare datasets

Have to split into datasets for variable WN
Have to separate by number of parameters, then join

In [ ]:
print(pardict)

In [ ]:
rescaled_11yr_chain_dict_comb = nltm.get_combined_rescaled_chain_dict(trimmed_chain_dict,pardict,'11yr')
az_comb_11yr_rescaled = az.convert_to_inference_data(rescaled_11yr_chain_dict_comb)

In [ ]:
rescaled_11yr_core_list_comb = nltm.get_rescaled_chains_as_core_list(rescaled_11yr_chain_dict_comb,psr_name)
eleven_yr_chaindirs = []
for chaindir in chaindir_list:
    if '11yr' in chaindir.split('/'):
        eleven_yr_chaindirs.append(chaindir)

In [ ]:
rescaled_12p5yr_chain_dict_comb = nltm.get_combined_rescaled_chain_dict(trimmed_chain_dict,pardict,'12p5yr')
az_comb_12p5yr_rescaled = az.convert_to_inference_data(rescaled_12p5yr_chain_dict_comb)

In [ ]:
rescaled_12p5yr_core_list_comb = nltm.get_rescaled_chains_as_core_list(rescaled_12p5yr_chain_dict_comb,psr_name)
twelvep5_yr_chaindirs = []
for chaindir in chaindir_list:
    if '12p5yr' in chaindir.split('/'):
        twelvep5_yr_chaindirs.append(chaindir)

Combine the common parameters for the not donked up datasets

In [ ]:
rescaled_chain_dict_all = {}
for param in rescaled_5yr_chain_dict_comb.keys():
    if param not in ['lnlike','lnprior', 'chain accept', 'pt chain accept']:
        if param in rescaled_9yr_chain_dict_comb.keys():
            if param in rescaled_chain_dict_all.keys():
                rescaled_chain_dict_all[param] = np.concatenate((rescaled_chain_dict_all[param],
                                                                 rescaled_9yr_chain_dict_comb[param]),axis=0)
            else:
                rescaled_chain_dict_all[param] = np.concatenate((rescaled_5yr_chain_dict_comb[param],
                                                                 rescaled_9yr_chain_dict_comb[param]),axis=0)
        if param in rescaled_11yr_chain_dict_comb.keys():
            if param in rescaled_chain_dict_all.keys():
                rescaled_chain_dict_all[param] = np.concatenate((rescaled_chain_dict_all[param],
                                                                 rescaled_11yr_chain_dict_comb[param]),axis=0)
            else:
                rescaled_chain_dict_all[param] = np.concatenate((rescaled_5yr_chain_dict_comb[param],
                                                                  rescaled_11yr_chain_dict_comb[param]),axis=0)
        if param in rescaled_12p5yr_chain_dict_comb.keys():
            if param in rescaled_chain_dict_all.keys():
                rescaled_chain_dict_all[param] = np.concatenate((rescaled_chain_dict_all[param],
                                                                 rescaled_12p5yr_chain_dict_comb[param]),axis=0)
            else:
                rescaled_chain_dict_all[param] = np.concatenate((rescaled_5yr_chain_dict_comb[param],
                                                                  rescaled_12p5yr_chain_dict_comb[param]),axis=0)

In [ ]:
#az.summary(az_comb_5yr_rescaled)

In [ ]:
#az.summary(az_comb_9yr_rescaled)

In [ ]:
def plt_all_rescaled_dataset(core_list,chaindir_list,dataset):
    core_kwargs = {'hist':True, 'ncols':4, 'title_y':1.03,
               'suptitle':'{} {} All Runs'.format(psrlist[0],dataset),
               'bins':10,'hist_kwargs':dict(fill=False),'linewidth':2.}
    misc_kwargs = {'legend_on':True,'legend_loc':(.7,0.05)}
    priors = []
    px_priors = []
    for p in chain_dict.keys():
        for px_p in chain_dict[p].keys():
            if dataset in chain_dict[p][px_p].keys():
                if p not in priors:
                    priors.append(p)
                if px_p not in px_priors:
                    px_priors.append(px_p)
    if priors and px_priors:
        nltm.plot_common_chains(core_list,chaindir_list,priors,px_priors,dataset,
                                plot_kwargs=core_kwargs,misc_kwargs=misc_kwargs)
    else:
        print(selection,'not in',dataset)

## 5yr Plots

`la_forge` plots, then `arviz`

In [ ]:
#All Runs
plt_all_rescaled_dataset(rescaled_5yr_core_list_comb,five_yr_chaindirs,'5yr')

In [ ]:
white_pars_5yr = []
parfile_priors_pars_5yr = []
for param in rescaled_5yr_chain_dict_comb.keys():
    if param not in ['lnlike','lnprior', 'chain accept', 'pt chain accept']:
        if param.split('_')[-1] in ['efac','equad','ecorr']:
            white_pars_5yr.append(param)
        else:
            parfile_priors_pars_5yr.append(param)

White Noise Parameters

In [ ]:
az.plot_trace(az_comb_5yr_rescaled,var_names=white_pars_5yr,plot_kwargs={'linewidth':3});

Red + Timing Parameters

In [ ]:
az.plot_trace(az_comb_5yr_rescaled,var_names=parfile_priors_pars_5yr,plot_kwargs={'linewidth':3});

## 9yr Plots

`la_forge` plots, then `arviz`

In [ ]:
#All Runs
plt_all_rescaled_dataset(rescaled_9yr_core_list_comb,nine_yr_chaindirs,'9yr')

In [ ]:
white_pars_9yr = []
parfile_priors_pars_9yr = []
for param in rescaled_9yr_chain_dict_comb.keys():
    if param not in ['lnlike','lnprior', 'chain accept', 'pt chain accept']:
        if param.split('_')[-1] in ['efac','equad','ecorr']:
            white_pars_9yr.append(param)
        else:
            parfile_priors_pars_9yr.append(param)

White noise parameters

In [ ]:
az.plot_trace(az_comb_9yr_rescaled,var_names=white_pars_9yr,plot_kwargs={'linewidth':3});

Red + Timing Parameters


In [ ]:
az.plot_trace(az_comb_9yr_rescaled,var_names=parfile_priors_pars_9yr,plot_kwargs={'linewidth':3});

## 11yr Plots

`la_forge` plots, then `arviz`

In [ ]:
#All Runs
plt_all_rescaled_dataset(rescaled_11yr_core_list_comb,eleven_yr_chaindirs,'11yr')

In [ ]:
white_pars_11yr = []
parfile_priors_pars_11yr = []
for param in rescaled_11yr_chain_dict_comb.keys():
    if param not in ['lnlike','lnprior', 'chain accept', 'pt chain accept']:
        if param.split('_')[-1] in ['efac','equad','ecorr']:
            white_pars_11yr.append(param)
        else:
            parfile_priors_pars_11yr.append(param)

White noise parameters

In [ ]:
az.plot_trace(az_comb_11yr_rescaled,var_names=white_pars_11yr,plot_kwargs={'linewidth':3});

Red + Timing Parameters

In [ ]:
az.plot_trace(az_comb_11yr_rescaled,var_names=parfile_priors_pars_11yr,plot_kwargs={'linewidth':3});

## 12.5yr Plots

`la_forge` plots, then `arviz`

In [ ]:
#All Runs
plt_all_rescaled_dataset(rescaled_12p5yr_core_list_comb,twelvep5_yr_chaindirs,'12p5yr')

In [ ]:
white_pars_12p5yr = []
parfile_priors_pars_12p5yr = []
for param in rescaled_12p5yr_chain_dict_comb.keys():
    if param not in ['lnlike','lnprior', 'chain accept', 'pt chain accept']:
        if param.split('_')[-1] in ['efac','equad','ecorr']:
            white_pars_12p5yr.append(param)
        else:
            parfile_priors_pars_12p5yr.append(param)

White noise parameters

In [ ]:
az.plot_trace(az_comb_11yr_rescaled,var_names=white_pars_11yr,plot_kwargs={'linewidth':3});

Red + Timing Parameters

In [ ]:
az.plot_trace(az_comb_11yr_rescaled,var_names=parfile_priors_pars_11yr,plot_kwargs={'linewidth':3});

## Combined All Plots

In [ ]:
az.plot_trace(rescaled_chain_dict_all,legend=True,figsize=(15,2*len(rescaled_chain_dict_all.keys())),
             plot_kwargs={'linewidth':3});

In [ ]:
#All Runs
core_kwargs = {'hist':True, 'ncols':4, 'title_y':1.03,
               'suptitle':'{} Combined 5yr+9yr+11yr All Runs'.format(psrlist[0]),
               'bins':10,'hist_kwargs':dict(fill=False),'linewidth':2.}
misc_kwargs = {'legend_on':True,'legend_loc':(.55,.2)}

datasets = ['5yr','9yr','11yr','12p']
priors = []
px_priors = []
for p in chain_dict.keys():
    for px_p in chain_dict[p].keys():
        for dataset in datasets:
            if dataset in chain_dict[p][px_p].keys():
                if p not in priors:
                    priors.append(p)
                if px_p not in px_priors:
                    px_priors.append(px_p)

nltm.plot_common_chains((rescaled_5yr_core_list_comb + rescaled_9yr_core_list_comb + \
                         rescaled_11yr_core_list_comb + rescaled_12p5yr_core_list_comb),
                        (five_yr_chaindirs+nine_yr_chaindirs+eleven_yr_chaindirs+twelvep5_yr_chaindirs),
                        priors,px_priors,datasets,
                        plot_kwargs=core_kwargs,misc_kwargs=misc_kwargs)

# PX Only

In [ ]:
tmp = rescaled_5yr_core_list_comb + rescaled_9yr_core_list_comb +\
rescaled_11yr_core_list_comb + rescaled_12p5yr_core_list_comb
px_array = []
for tmp_core in tmp:
    px_array.append(tmp_core.get_param('PX',to_burn=False))

In [ ]:
mus = []
lb = []
ub = []
for dataset in pardict.keys():
    for parvals in pardict[dataset].values():
        mus.append(parvals['PX']['val'])
        lb.append(parvals['PX']['val']-parvals['PX']['val']*parvals['PX']['err'])
        ub.append(parvals['PX']['val']+parvals['PX']['val']*parvals['PX']['err'])

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(15,7.5))
hist_kwargs = {'fill':False,'bins':20,'linewidth':3.,'density':True,'histtype':'step'}
legend_list = ['5yr: Uniform, Parfile Priors','5yr: Bounded Normal, Parfile Priors',
               '9yr: Uniform, Parfile Priors','9yr: Bounded Normal, Parfile Priors',
               '11yr: Uniform, Parfile Priors','11yr: Bounded Normal, Parfile Priors']

for i,px in enumerate(px_array):
    hist_kwargs['Color'] = 'C{}'.format(i)
    #hist_kwargs['label'] = legend_list[i]
    ax.hist(px,**hist_kwargs);

legend_list_2 = ['5yr Parfile PX','9yr Parfile PX','11yr Parfile PX','12.5yr Parfile PX']
for i,mu in enumerate(mus):
    ax.axvline(x=mu,linestyle='--',linewidth=3,color='C{}'.format(i),label=legend_list_2[i]+': %.3f' %mu)
ax.set_xlabel(r'Parallax (mas)',fontsize=18)
plt.title('{} Parallax Comparison of 5yr, 9yr, 11yr, and 12.5yr Runs'.format(psrlist[0]))
plt.legend()#loc=(.48,.7))
#plt.savefig('./Figures/{}_px_comp.png'.format(psrlist[0]),dpi=300,bbox_inches='tight')
plt.show()